# 数据预处理

## 选取所需要的数据(筛选字段)

In [161]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
usecols = ['选购商品','商品规格','商品数量','商品单价','订单应付金额','支付方式',
'省','市','APP渠道','流量来源','订单状态','售后状态','广告渠道','流量体裁','流量渠道',
'订单提交时间','订单完成时间','支付完成时间'] 
#'降价类优惠','是否安心购','旗帜颜色','平台优惠','发货时间'
len(usecols)

18

In [162]:
# data_7: 2023-05-16到2023-05-23
# data_15:2023-05-01到2023-05-15
# data_30:2023-04-01到2023-04-30

In [163]:
data_7 = pd.read_csv('./近7天订单数据.csv',encoding='gbk',usecols=usecols)
data_15 = pd.read_csv('./上半月订单数据.csv',encoding='gbk',usecols=usecols)
data_30 = pd.read_csv('./上月订单数据.csv',encoding='gbk',usecols=usecols)

## 数据格式转换

In [164]:
#删除一些字段前后的空格
for i in ['选购商品','商品规格','订单提交时间','支付方式','订单完成时间','支付完成时间','订单状态']:#,'发货时间'
    data_7[i] = data_7[i].str.strip()
    data_15[i] = data_15[i].str.strip()
    data_30[i] = data_30[i].str.strip()

#把时间从字符串转换为日期时间类型
for j in ['订单提交时间','订单完成时间','支付完成时间']:#,'发货时间']:
    data_7[j] = pd.to_datetime(data_7[j])
    data_15[j] = pd.to_datetime(data_15[j])
    data_30[j] = pd.to_datetime(data_30[j])

## 数据拼接

In [165]:
#2023-04-01到2023-05-15的数据
data_45 = pd.concat([data_15,data_30],axis = 0)
#所有数据
data_52 = pd.concat([data_7,data_15,data_30],axis = 0)
#重置索引
data_45.reset_index(inplace=True)
data_52.reset_index(inplace=True)

## 成功付款的订单数据

In [166]:
# 成功付款的订单
data_1 = data_52[data_52['订单状态']!='已关闭']
data_suss = data_1[data_1['订单状态']!='待支付']

# 近七日与一个半月前对比的统计分析

## 日均销售额

In [167]:
# 日均销售额
sales7 = data_7['订单应付金额'].sum()
sales45 = data_45['订单应付金额'].sum()
print(f'近七天的日均支付额:{int(sales7/7)},过去一个半月的日均销售额是:{int((sales45)/45)},下降了{int((sales45/45)-(sales7/7))}')

近七天的日均支付额:166381,过去一个半月的日均销售额是:193499,下降了27118


## 售后发起率

In [168]:
# 售后发起率
after_sales7 = ((data_7.shape[0]-data_7['售后状态'].value_counts()['-'])/data_7.shape[0])
after_sales30 = ((data_30.shape[0]-data_30['售后状态'].value_counts()['-'])/data_30.shape[0])
after_sales45 = ((data_45.shape[0]-data_45['售后状态'].value_counts()['-'])/data_45.shape[0])

In [169]:
print(f'近七天的售后发起率:{after_sales7:.3%},过去一个半月的售后发起率是:{after_sales45:.3%},下降了{after_sales45-after_sales7:.3%}')

近七天的售后发起率:8.176%,过去一个半月的售后发起率是:10.442%,下降了2.265%


# 整体性统计分析

## 所卖商品数量、总金额、客单价

In [170]:
numb = (data_suss['商品数量'].sum())
amount = data_suss['订单应付金额'].sum()
unit_price = amount/numb
print(f'所卖商品数量:{numb}、总金额:{amount}、客单价:{unit_price:.2f}')

所卖商品数量:108047、总金额:6864589.58、客单价:63.53


## 查看下单量前十、倒十的商品

In [171]:
# 红榜
data_52['选购商品'].value_counts()[:10] 

LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉            45837
【限量特惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉    36023
【5根金属支撑条】支撑透气护腰带，四季通用，支撑减压 束腰收腹             26142
【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉    18112
【买一送二】新一代保暖护腰，双面磁石保暖，四根钢板支撑减压               10423
【居家健身必备】11件综合训练拉力绳,一套搞定全身肌肉,4根80磅            2445
LIARJAN/莱尔健计数握力器 不同握力阶段人群的锻炼神器 臂肌             573
LIARJAN/莱尔健【升级】家庭综合训练拉力绳，一套搞定全身肌肉             411
专业级全能运动护膝 男女膝盖篮球装备 关节半月板保护套跑步                 247
LIARJAN/莱尔健「麒麟臂专用」拉力绳+握力器 强化力量 智能计数           196
Name: 选购商品, dtype: int64

In [172]:
# 黑榜
data_52['选购商品'].value_counts()[-10:] 

LIARJAN/莱尔健「减肚子必备」拉力绳+呼啦圈暴汗燃脂腹肌马甲线    3
LIARJAN/莱尔健呼啦圈 收腹减肥燃脂神器 专业美腰瘦腰肚子      3
专业级全能运动护膝，保护爱运动的你                     3
LIARJAN/莱尔健【基础】家庭综合训练拉力绳，一套练全身肌肉      2
LIARJAN/莱尔健筋膜枪肌肉放松按摩锤成年人解压瘦腰腿健身器材     2
腹部拉力器 脚蹬腹部腰部训练器材家用 拉力绳拉力器腹肌马甲线        2
LIARJAN/莱尔健「增肌+减脂组合」拉力绳+跳绳            2
【只卖质量不卖价格】自动回弹腹肌肘撑健腹轮训练器男女健身塑形        2
颈椎按摩枕重力指压颈部按摩器枕颈肩颈椎家用按摩枕头牵引矫正器        1
【第三代】瑜伽夹腿训练器 多功能夹腿健身瑜伽健身夹腿器           1
Name: 选购商品, dtype: int64

In [173]:
# 商家应该与厂商沟通，重点备货销量以及销售规格最高的那些商品

## 分析金额前五十的订单

In [174]:
# 付款金额前五十的订单
order_50 = data_suss.sort_values('订单应付金额')[-50:]
order_50['广告渠道'].value_counts()
# 可见订单金额前五十的订单中直播下单的达到了一半
# 应投入更多于直播中

直播    25
无     25
Name: 广告渠道, dtype: int64

In [175]:
order_50['流量来源'].value_counts()
# 精选联盟是重大的流量来源，应好好把握

精选联盟    36
小店自卖    12
-        2
Name: 流量来源, dtype: int64

## 分析异常订单数据

In [176]:
data_52[data_52['订单应付金额'] == 812][data_52['订单状态'] == '已关闭']
# 付款金额为812的订单通过excel筛选可以明显看出全部均取消(52天里共有1158条)
# 可以把该类用户进行拉黑处理(同一个地址下单多次、锁单且不付款的，下单时显示下单失败)

,index,选购商品,商品规格,商品数量,商品单价,订单应付金额,支付方式,省,市,订单提交时间,订单完成时间,支付完成时间,APP渠道,流量来源,订单状态,售后状态,广告渠道,流量体裁,流量渠道
168,168,【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,支付宝,广东省,广州市,2023-05-23 15:03:51,NaT,NaT,抖音,精选联盟,已关闭,-,无,数据将于第二天更新,数据将于第二天更新
169,169,【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,支付宝,广东省,广州市,2023-05-23 15:03:24,NaT,NaT,抖音,精选联盟,已关闭,-,无,数据将于第二天更新,数据将于第二天更新
170,170,【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,支付宝,广东省,广州市,2023-05-23 15:03:15,NaT,NaT,抖音,精选联盟,已关闭,-,无,数据将于第二天更新,数据将于第二天更新
171,171,【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,支付宝,广东省,广州市,2023-05-23 15:03:07,NaT,NaT,抖音,精选联盟,已关闭,-,无,数据将于第二天更新,数据将于第二天更新
172,172,【限量优惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,支付宝,广东省,广州市,2023-05-23 15:02:58,NaT,NaT,抖音,精选联盟,已关闭,-,无,数据将于第二天更新,数据将于第二天更新
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140056,89314,【限量特惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,抖音月付,广东省,广州市,2023-04-01 10:34:06,NaT,NaT,抖音,精选联盟,已关闭,-,无,直播,关注来源
140057,89315,【限量特惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,抖音月付,广东省,广州市,2023-04-01 10:33:55,NaT,NaT,抖音,精选联盟,已关闭,-,无,直播,关注来源
140058,89316,【限量特惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,抖音月付,广东省,广州市,2023-04-01 10:33:46,NaT,NaT,抖音,精选联盟,已关闭,-,无,直播,关注来源
140060,89318,【限量特惠专属】LIARJAN/莱尔健【标准】家庭综合训练拉力绳，一套练全身肌肉,【女生款】女生体重＜50kg：9件套2根绳40磅=29元,28,29.0,812.0,抖音月付,广东省,广州市,2023-04-01 10:33:34,NaT,NaT,抖音,精选联盟,已关闭,-,无,直播,关注来源


# 时间转化分析

## 下单到付款所用的用时

In [177]:
#总的平均付款所用时间
data_suss['付款所用时间'] = data_suss['支付完成时间'] - data_suss['订单提交时间']
data_suss['付款所用时间'].sum() / data_suss['付款所用时间'].shape[0]

Timedelta('0 days 00:00:17.947233853')

In [178]:
#获取支付完成时间
data_suss['月份'] = data_suss['支付完成时间'].dt.month
data_suss['天数'] = data_suss['支付完成时间'].dt.day

In [179]:
#从4-01作为第一天，5-23作为第53天得到天数
data_suss['第x天'] = (data_suss['月份']-4)*30 + data_suss['天数']

In [180]:
time = data_suss.groupby(data_suss['第x天']).agg({'付款所用时间':'sum'})
times = (data_suss.groupby(data_suss['第x天']).agg({'付款所用时间':'count'}))

In [181]:
tim = pd.DataFrame()
tim['总用时'] = time
tim['总次数'] = times
tim['平均付款用时'] = tim['总用时']/tim['总次数']#.dt.total_seconds()
tim
# 可见付款成功订单平均用时都在16-21秒左右，属于正常范围，可以预测用户下单时遇到网络不流畅、服务器卡顿的可能性较小

,总用时,总次数,平均付款用时
第x天,,,
1,0 days 15:49:42,2785,0 days 00:00:20.460323159
2,0 days 06:08:00,1226,0 days 00:00:18.009787928
3,0 days 09:49:13,1726,0 days 00:00:20.482618771
4,0 days 13:14:14,2553,0 days 00:00:18.665883274
5,0 days 06:06:13,1059,0 days 00:00:20.748819641
6,0 days 13:02:56,2396,0 days 00:00:19.606010016
7,0 days 13:40:16,2179,0 days 00:00:22.586507572
8,0 days 18:54:05,3834,0 days 00:00:17.747782994
9,0 days 13:14:32,2107,0 days 00:00:22.625533934


# 退款数据分析

In [182]:
# data_refund：开启售后的订单
data_refund = data_52[data_52['售后状态'] != '-']
data_refund['流量体裁'].value_counts()

短视频          7257
直播           6421
商品卡           400
其他            155
数据将于第二天更新      80
Name: 流量体裁, dtype: int64

In [183]:
data_52['流量体裁'].value_counts()

短视频          77258
直播           55969
商品卡           3919
其他            2199
数据将于第二天更新     1436
Name: 流量体裁, dtype: int64

In [184]:
refund = pd.DataFrame()
refund['总单量'] = data_refund['流量体裁'].value_counts()
refund['总退货数'] = data_52['流量体裁'].value_counts()
refund['渠道退货率'] = ((refund['总单量']/refund['总退货数'])*100).round(2).astype(str)+'%'
refund
# 短视频、直播、商品卡、退货率都较为高，可增加这三个场景商品描述的真实性，减少顾客不必要退货带来的商家运费亏损

,总单量,总退货数,渠道退货率
短视频,7257,77258,9.39%
直播,6421,55969,11.47%
商品卡,400,3919,10.21%
其他,155,2199,7.05%
数据将于第二天更新,80,1436,5.57%


# 决策建议

## 查看地区的订单情况

In [185]:
def area(x):
    if x in ['广东省','广西壮族自治区','海南省','福建省']:
        return '华南地区'
    elif x in ['湖北省','湖南省','河南省','江西省']:
        return '华中地区'
    elif x in ['山东省','江苏省','安徽省','浙江省','上海市']:
        return '华东地区'
    elif x in ['北京市','天津市','河北省','山西省','内蒙古自治区']:
        return '华北地区'
    elif x in ['黑龙江省','吉林省','辽宁省']:
        return '东北三省'
    elif x in ['陕西省','甘肃省','宁夏回族自治区','青海省','新疆维吾尔自治区']:
        return '西北地区'
    elif x in ['云南省','贵州省','重庆市','四川省','西藏藏族自治区']:
        return '西南地区'
    else:
        pass

In [186]:
data_52['所属地区'] = data_52['省'].apply(area)
data_52['所属地区'].value_counts()
# 购买地址主要集中在华东华南华中一带地区
# 所以可以主要在三个地区的折中交汇处设立发货仓库(如江西-福建一带)来减小物流成本。

华东地区    46183
华南地区    27186
华中地区    20096
西南地区    17335
华北地区    15120
西北地区     7983
东北三省     6877
Name: 所属地区, dtype: int64

## 查看用户活跃的时间段并投入更多于直播

In [187]:
data_52['订单提交时间'].dt.hour.value_counts()[:7]
# 11.12.13.14 19.20.21   
# 这些时段明显活跃用户数很高,建议在这些时间段可以加大直播运营的投入
# 让那些犹豫下单的用户对产品有更加全面的了解,提高成单率
# 用最低的成本让最更的用户了解产品

12    11704
20    10006
11     9535
13     9031
19     8396
14     8264
21     7253
Name: 订单提交时间, dtype: int64